In [26]:
#from nipype.interfaces import niftyreg
import numpy as np
from skimage import io
from pathlib import Path
import re
import ants
from skimage.transform import resize
from tqdm import tqdm
from skimage.morphology import skeletonize_3d, binary_dilation, binary_closing
from scipy.ndimage import distance_transform_edt
import tifffile as tif
from scipy.ndimage import binary_fill_holes 
import cc3d
from scipy.io import loadmat, savemat
import skan
import sknw
import networkx as nx
import pickle
import os
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt

# Define connected componnet removal

In [2]:
def remove_small_comps_3d(image, thresh = 500):
    """
    

    Parameters
    ----------
    image : binary np array with uint8 elements
        3d numpy matrix, connected components will be removed form this image
    thresh : int64
        smallest connected components to keep

    Returns
    -------
    np.array with uint8 elements, binary
        binary image with connected components below the threshold removed.

    """
    img_lab, N = cc3d.connected_components(image,return_N=True)
    unique, counts = np.unique(img_lab, return_counts=True)
    unique_keep = unique[counts>thresh]
    unique_keep = np.delete(unique_keep,[0])
    img_filt = np.zeros(img_lab.shape).astype('int8')
    img_filt[np.isin(img_lab,unique_keep)] = 1
    return img_filt.astype('uint8')   

def fill_holes(img,thresh=100):
    #res = np.zeros(img.shape)
    for i in np.unique(img)[::-1]:
        _tmp = (img==i)*1.0
        _tmp = _tmp.astype('int8')
        _tmp = remove_small_comps_3d(_tmp,thresh=thresh)
        img[_tmp==1] = i
    res = img.astype('int8')
    return res
def invert_raw(x):
    return x[:,:,:,::-1]
def invert_label(x):
    return x[:,:,::-1]

# Get mean predictions

In [20]:
directory = Path('matt_preds')
files  = directory.glob('*_mean.npy')
files = sorted([x.as_posix() for x in files])

In [21]:
len(files)

390

# High bias low varience segmentation

With removal of connected components under 500 pixels

In [28]:
min_prob = 0.75
max_var = 0.1
for file in tqdm(files[18:]):
    if not dt.datetime.fromtimestamp(os.path.getctime(re.sub('mean','seg',file))).date() == dt.datetime.now().date():
    #if not os.path.exists(re.sub('mean','seg',file)):
        mean = np.load(file)
        std = np.load(re.sub('mean','std',file))
        seg = np.zeros(mean.shape[1:])
        seg[(mean[1,:,:,:] > min_prob) * (std[1,:,:,:] < max_var)] = 1
        seg[(mean[2,:,:,:] > min_prob) * (std[2,:,:,:] < max_var)] = 2
        seg = seg.astype('int8')
        seg = (seg==1)*1
        seg = fill_holes(seg)
        seg = seg.astype('float32')
        np.save(re.sub('mean','seg',file),seg)
        #savemat(re.sub('mean.npy','seg.mat',file),{'FinalImage':fill_holes(binary_dilation(binary_dilation(seg)))})
        #tif.imwrite(re.sub('mean.npy','seg.tif',file),seg)

100%|██████████| 372/372 [29:38<00:00,  4.78s/it] 


# ANTs registration

## Upsample raw images

In [29]:
image_path = Path('../TBI')
images = list(image_path.glob('*_*/XYZres*?[0-9].tif'))
images = sorted([x.as_posix() for x in images])
len(images)

432

In [30]:
for i in tqdm(range(len(images))[::-1]):
    if not os.path.exists('matt_preds/'+re.sub('.tif','_upsamp.npy',Path(images[i]).name)):
        image = io.imread(images[i])
        image = np.swapaxes(image,0,1)
        image = np.swapaxes(image,1,3)
        image = resize(image,(2,507,507,252),preserve_range=True)
        zeros = np.zeros((1,507,507,252)).astype('uint16')
        image = np.append(image,zeros,axis = 0)
        image = image.astype('float32')
        io.imsave('matt_preds/'+re.sub('.tif','_upsamp.tif',Path(images[i]).name),image)
        np.save('matt_preds/'+re.sub('.tif','_upsamp.npy',Path(images[i]).name),image)

100%|██████████| 432/432 [00:00<00:00, 36095.85it/s]


In [31]:
images[i]

'../TBI/20200217_11/XYZres100.tif'

## reshape numpy seg files and resave as tif

In [32]:
directory_seg = Path('matt_preds')
files_seg  = list(directory_seg.glob('*_seg.npy'))
files_seg = sorted([x.as_posix() for x in files_seg])

In [33]:
for i in tqdm(range(len(files_seg))):
    if not os.path.exists(re.sub('.npy','.tif',files_seg[i])):
        tmp = np.load(files_seg[i])
        tmp = np.reshape(tmp,(1,507,507,252))
        io.imsave(re.sub('.npy','.tif',files_seg[i]),tmp.astype('float32'))

100%|██████████| 390/390 [00:00<00:00, 69026.02it/s]


## Register images and transform masks

In [42]:
rep = {
    "53":"60",
    "54":"59",
    "55":"58",
    "56":"57",
    "69":"76",
    "70":"75",
    "71":"74",
    "72":"73",
    "85":"88",
    "86":"90",
    "95":"102",
    "96":"101",
    "97":"100",
    "98":"99",
    "103":"110",
    "104":"109",
    "106":"111",
    "112":"109",
    "115":"118",
    "116":"117",
    "117":"120",
    "125":"132",
    "126":"131",
    "127":"130",
    "128":"129",
    "143":"149",
    "144":"150",
    "145":"147",
    "146":"148",
    "151":"158",
    "154":"157",
    "159":"166",
    "160":"165",
    "161":"164",
    "162":"163",
    "171":"178",
    "172":"177",
    "173":"176",
    "174":"175",
    "187":"190",
    "184":"189",
    "185":"188",
    "186":"191",
    "198":"205",
    "199":"204",
    "200":"203",
    "201":"202",
    "210":"213",
    "216":"211",
    "218":"217",
    "212":"215",
    "237":"240",
    "235":"234",
    "240":"241",
    "233":"236",
    "244":"249",
    "248":"252",
    "250":"243",
    "247":"246",
    "253":"260",
    "261":"256",
    "255":"258",
    "262":"254",
    "264":"272",
    "266":"267",
    "270":"263",
    "272":"286",
    "278":"282",
    "273":"275",
    "277":"280",
    "283":"286",
    "284":"285",
    "253":"260",
    "262":"254",
    "300":"288",
    "294":"298",
    "297":"295",
    "329":"332",
    "338":"334",
    "333":"336",
    "335":"330"
}

In [48]:
directory_seg = Path('matt_preds')
images = list(directory_seg.glob('*res*_upsamp.npy'))
images = sorted([x.as_posix() for x in images if '_0001' not in x.as_posix()])#[0:2]
images = [x for x in images if any(y in x for y in rep.keys())]
images[0]

'matt_preds/XYZres103_upsamp.npy'

In [49]:
images = [x for x in images if any(y in x for y in rep.keys())][::-1]
for x in rep.keys():
    if x in images[0]:
        print(re.sub(x,rep[x],images[0]))

matt_preds/XYZres99_upsamp.npy


In [50]:
for i in tqdm(range(len(images))):
    if os.path.exists(re.sub('upsamp','seg',images[i])):
        if not os.path.exists(re.sub('seg.npy','seg_warped_baseline.tif',re.sub('upsamp','seg',images[i]))):
            for x in rep.keys():
                if x in images[i]:
                    _new_image = re.sub(x,rep[x],images[i])
            print(_new_image)
            fix = ants.image_read(_new_image)
            mov = ants.image_read(images[i]).apply(invert_raw)
            fix_mask = ants.image_read(re.sub('upsamp','seg',_new_image))
            mov_mask = ants.image_read(re.sub('upsamp','seg',images[i])).apply(invert_label)
            mytx = ants.registration(fixed = fix,
                                    moving = mov,
                                    type_of_transform = 'Rigid'
                                    )
            warpedmask = ants.apply_transforms(fixed = fix_mask,
                                               moving = mov_mask,
                                               transformlist = mytx['fwdtransforms'],
                                               interpolator = 'nearestNeighbor'
                                              )
            io.imsave(re.sub('seg.npy','seg_warped_baseline.tif',re.sub('upsamp','seg',images[i])),warpedmask.numpy().astype('int8'))
            io.imsave(re.sub('seg.npy','seg_warped_baseline.tif',re.sub('upsamp','seg',_new_image)),fix_mask.numpy().astype('int8'))
            #print(2 * np.sum(warpedmask.numpy()*fix_mask.numpy())/(np.sum(warpedmask.numpy())+np.sum(fix_mask.numpy())))

  0%|          | 0/67 [00:00<?, ?it/s]

matt_preds/XYZres100_upsamp.npy


/tmp/ipykernel_2318938/3636203856.py:21: UserWarning:

matt_preds/XYZres97_seg_warped_baseline.tif is a low contrast image

/tmp/ipykernel_2318938/3636203856.py:22: UserWarning:

matt_preds/XYZres100_seg_warped_baseline.tif is a low contrast image

  3%|▎         | 2/67 [01:05<35:33, 32.82s/it]

matt_preds/XYZres101_upsamp.npy


/tmp/ipykernel_2318938/3636203856.py:21: UserWarning:

matt_preds/XYZres96_seg_warped_baseline.tif is a low contrast image

/tmp/ipykernel_2318938/3636203856.py:22: UserWarning:

matt_preds/XYZres101_seg_warped_baseline.tif is a low contrast image

  4%|▍         | 3/67 [02:13<50:39, 47.49s/it]

...

/tmp/ipykernel_2318938/3636203856.py:21: UserWarning:

matt_preds/XYZres199_seg_warped_baseline.tif is a low contrast image

/tmp/ipykernel_2318938/3636203856.py:22: UserWarning:

matt_preds/XYZres204_seg_warped_baseline.tif is a low contrast image

 43%|████▎     | 29/67 [34:09<47:18, 74.70s/it]

matt_preds/XYZres205_upsamp.npy


/tmp/ipykernel_2318938/3636203856.py:21: UserWarning:

matt_preds/XYZres198_seg_warped_baseline.tif is a low contrast image

/tmp/ipykernel_2318938/3636203856.py:22: UserWarning:

matt_preds/XYZres205_seg_warped_baseline.tif is a low contrast image

 45%|████▍     | 30/67 [35:26<43:42, 70.88s/it]


matt_preds/XYZres1100_upsamp.npy


FileNotFoundError: [Errno 2] No such file or directory: 'matt_preds/XYZres1100_upsamp.npy'

In [ ]:
fix_mask = ants.image_read(re.sub('upsamp','seg',_new_image))
mov_mask = ants.image_read(re.sub('upsamp','seg',images[i]))
np.sum(fix_mask.numpy()[:,:,::-1]*mov_mask.numpy())

In [ ]:
fix_mask

In [ ]:
ants.image_read(images[0]).apply(invert)

## Save Matlab .mat file of registered images

In [ ]:
directory_seg = Path('matt_preds')
images = list(directory_seg.glob('*_seg_warped_baseline.tif'))
images = sorted([x.as_posix() for x in images])[::-1]
images = [x for x in images if any(y in x for y in rep.keys())]
images

In [ ]:
for image in tqdm(images):
    #if not os.path.exists(re.sub('_seg_warped_baseline.tif','_seg_warped_baseline_single.mat',image)):
        for x in rep.keys():
                if x in image:
                    _new_image = re.sub(x,rep[x],image)
        img_0001 = io.imread(_new_image)
        img = io.imread(image)
        seg = img*img_0001
        seg = (seg==1)*1
        print(np.sum(seg))
        seg = seg.astype('int8')
        seg = fill_holes(seg)
        savemat(re.sub('_seg_warped_baseline.tif','_seg_warped_baseline_single.mat',image),{'FinalImage':fill_holes(binary_dilation(seg))})
        
        

In [ ]:
re.sub('_seg_warped_baseline.tif','_seg_warped_baseline_single.mat',image)

# Generate Graphs

In [ ]:
directory = Path('../TBI_monai_UNET/james_preds')
files_seg_0001 = directory.glob('*_seg_warped_baseline.tif')
files_seg_0001 = sorted([x.as_posix() for x in files_seg_0001])
files_seg_0001 = [x for x in files_seg_0001 if any(y in x for y in rep.keys())]
files_seg_0001

In [ ]:
for file_0001 in tqdm(files_seg_0001):
    file = file_0001
    skel_file = re.sub('_seg_warped_baseline.tif','_skel_warped_single_baseline.mat',file)
    #seg = io.imread(file)
    skel = loadmat(skel_file)['FilteredImage']
    #skel = skeletonize_3d(skel)
    io.imsave(re.sub('_seg_warped_baseline.tif','_single_skel_baseline.tif',file),skel)
    #dst_tsfm = distance_transform_edt(seg)
    #dst_tsfm[dst_tsfm==0]=0.00001
    #skel_dst = skel*dst_tsfm
#
    #
    #np.save(re.sub('seg','dst_skel',file),skel_dst)
    #io.imsave(re.sub('_seg_warped.tif','_dst_skel_warped.tif',file),skel_dst)
    graph = sknw.build_sknw(skel, multi=False)
    print(len(graph.edges))
    #graph, c0  = skan.csr.skeleton_to_csgraph(skel)
    #print(len(graph.edges))
    
    #print(len(graph_0001.edges))
    #pickle.dump(graph, open(str(re.sub('_seg.npy','.pickle',file)), 'w'))
    #nx.write_pajek(graph,re.sub('_seg.npy','.pajek',file))
    
    nx.write_gpickle(graph,re.sub('_seg_warped_baseline.tif','_warped_baseline.pickle',file))

In [ ]:
re.sub('_seg_warped_baseline.tif','_skel_warped_single_baseline.mat',file)

# write vessel measurments to graph files

In [ ]:
directory = Path('../TBI_monai_UNET/james_preds')
files = directory.glob('*_warped_baseline.pickle')
files = sorted([x.as_posix() for x in files])
print(len(files))
files[0]

In [ ]:
re.sub('../TBI_monai_UNET/james_preds/','',re.sub('_warped_baseline.pickle','',file))

In [ ]:
i=0
xls = pd.ExcelFile('../TBI_monai_UNET/p3_metalog.xlsx')
df = {}
for sheet_name in xls.sheet_names:
    df[sheet_name] = xls.parse(sheet_name)

for file in tqdm(files):
    graph = nx.read_gpickle(file)
    if len(graph.edges) < 1500:
        seg_file = re.sub('_warped_baseline.pickle','_seg_warped_baseline.tif',file)
        print(seg_file)
        for x in rep.keys():
                if x in seg_file:
                    _new_image = re.sub(x,rep[x],seg_file)
        seg_0001_file = _new_image
        print(seg_0001_file)
        seg = io.imread(seg_file)
        seg_0001 = io.imread(seg_0001_file)
        seg_dst = distance_transform_edt(seg)
        seg_0001_dst = distance_transform_edt(seg_0001)
        for sheet_name in xls.sheet_names:
            if re.sub('../TBI_monai_UNET/james_preds/','',re.sub('_warped_baseline.pickle','',file)) in df[sheet_name].values:
                subj = sheet_name
                _tmp = df[subj].loc[df[subj]['CHECK WATER'] == re.sub('../TBI_monai_UNET/james_preds/','',re.sub('_warped_baseline.pickle','',file))]
                if _tmp['Unnamed: 12'].iloc[0] == 'raster':
                    wavelength = _tmp['Unnamed: 11'].iloc[0]
                    power_per = _tmp['Unnamed: 10'].iloc[0]
                    start_depth = _tmp['Unnamed: 2'].iloc[0]
                    for i in range(len(graph.edges)):
                        path = graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['pts']
                        _vals = seg_dst[path[::-1,0],path[::-1,1],path[::-1,2]]
                        _vals_0001 = seg_0001_dst[path[::-1,0],path[::-1,1],path[::-1,2]]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii'] = np.mean(_vals)
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii_std'] = np.std(_vals)
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii_0001'] = np.mean(_vals_0001)
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii_0001_std'] = np.std(_vals_0001)
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['delta'] = np.mean(_vals_0001) - np.mean(_vals)
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['path_weights'] = _vals
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['path_weights_0001'] = _vals_0001
                        #graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['weight'] = graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['weight']
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0z'] = path[0][0]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0y'] = path[0][1]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0x'] = path[0][2]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1z'] = path[-1][0]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1y'] = path[-1][1]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1x'] = path[-1][2]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['img_start_depth'] = start_depth
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['mean_depth'] = np.mean(path[:,0])
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['euclidean-dst'] = np.sqrt(np.sum(np.square(path[-1]-path[0])))
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['subject'] = subj
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['wavelength'] = wavelength
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['power'] = power_per
                    nx.write_gpickle(graph, re.sub('preds','preds_graphs',re.sub('.pickle','_radii.pickle',file)))
                    i+=1
                    print(i)
print(i)

In [ ]:
df[subj].loc[df[subj]['CHECK WATER'] == re.sub('../TBI_monai_UNET/james_preds/','',re.sub('_warped_baseline.pickle','',file))]

# convert graphs to excel files

In [ ]:
directory = Path('../TBI_monai_UNET/james_preds_graphs')
files = directory.glob('*_warped_baseline_radii.pickle')
files = sorted([x.as_posix() for x in files])
print(len(files))
files[0]

In [ ]:
for file in tqdm(files):
    graph = nx.read_gpickle(file)
    edge_df = nx.to_pandas_edgelist(graph)
    edge_df.to_excel(re.sub('.pickle','.xlsx',file))

In [ ]:
edge_df